<a href="https://colab.research.google.com/github/matteraggi/FineTuningAI/blob/main/ProgettoAggiornato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup iniziale

**INSTALLS**

In [ ]:
!pip install --upgrade transformers torch datasets peft tf-keras accelerate bitsandbytes trl evaluate radon zss torchvision
# IMPORTANTE! RIAVVIARE IL RUNTIME DOPO L'ESECUZIONE

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.1 MB/s eta 0:00:00
   ━━━━

**IMPORTS**

In [ ]:
# Importing stock ml libraries
import pandas as pd
import torch
import gc
from tabulate import tabulate
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
import json
import random
from google.colab import files

**Svuota la cache**

In [ ]:
# Garbace collect
gc.collect()
# Svuota la cache
torch.cuda.empty_cache()

**VARIABLES**

In [ ]:
# The model that you want to train from the Hugging Face hub
#model_name = "bigcode/starcoder2-3b"
#model_name = "openai-community/gpt2"
#model_name = "NousResearch/Llama-2-7b-hf"
#model_name = "deepcogito/cogito-v1-preview-llama-3B"
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
starcoder_name = "bigcode/starcoder2-3b"

# The instruction dataset to use
dataset_name = "bigcode/self-oss-instruct-sc2-exec-filter-50k"
# Fine-tuned model name
new_model = "Our_Finetuned_Model"

# Dimension of dataset subset used
dataset_range = 4000

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension. It determines the size and parameter count of the low-rank adaptation
lora_r = 64
# Alpha parameter for LoRA scaling factor that determines the impact of the low-rank matrices on the original model's output.
# Controls the overall strength of the low-rank adaptation.
lora_alpha = 2*lora_r  # "Often set to 2-4 times lora_r"
# Dropout probability for LoRA layers (considera aumento a 0.1-0.2)
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False  # "Double quantization can sometimes improve performance but increases complexity.  It's often left disabled initially."

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False  # CONFERMO CHE NON E' SUPPORTATO SU COLAB

# Number of training epochs
num_train_epochs = 3  # Sembra arrivare a un plateau già alla terza
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4  # Abbassare in caso di training più lunghi

# Batch size per training e per evaluation
per_device_train_batch_size = per_device_eval_batch_size = 2
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4 # Moltiplica la batch_size vera per ottenere quella simulata. Dinimuire per velocità, alzare per stabilità.

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3  # Stabilizza il training, ma rallentandolo. Range ottimale 0.1-0.5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.1  # Aumentare in caso di overfitting a 0.1-0.2

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1  # '-1' mantains num_train_epochs

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.05 # Aumentare a 0.1 in caso di instabilità eccessiva iniziale

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every tot updates steps
save_steps = 100
# Log every tot updates steps
logging_steps = 100
# Evaluation strategy ("no", "epochs", "steps")
eval_strategy = "steps" # unused
# Evaluate the model every tot steps (if the strategy is "steps")
eval_steps = 100        # unused

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use; tronca l'input a tot tokens
max_seq_length = 1024

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False  # "Can improve efficiency if your dataset has many short sequences."

# Load the entire model on the GPU 0
device_map = {"": 0}

################################################################################
# Parameters for code generation
################################################################################

# Maximum number of new tokens to generate
max_new_tokens=500
# Enable sampling for more diverse outputs
do_sample=True
# Temperature for sampling (0 to 1, higher = more random)
temperature=0.8
# Top-k sampling (0 to disable, higher = more diverse)
top_p=0.95
# Repetition penalty to apply to the generated text (1.0 = no penalty, >1.0 = penalizes repetition)
repetition_penalty=1.1

### Caricamento di DeepSeek (modello di base per il fine-tuning)

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)
# Il warning di sliding window è un false warning: https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/discussions/27

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config=bnb_config,
  device_map=device_map
)
base_model.eval()
# Load base tokenizer
tokenizer_base = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
print(tokenizer_base.pad_token, tokenizer_base.eos_token)
tokenizer_base.pad_token = tokenizer_base.eos_token
print(tokenizer_base.pad_token, tokenizer_base.eos_token)
tokenizer_base.padding_side = 'right'

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

<｜end▁of▁sentence｜> <｜end▁of▁sentence｜>
<｜end▁of▁sentence｜> <｜end▁of▁sentence｜>


In [ ]:
# Ultime configurazioni per il modello
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1
if gradient_checkpointing:
  base_model.gradient_checkpointing_enable()  # Attiva gradient checkpointing per ridurre l'uso di memoria

# for name, module in base_model.named_modules():
#    print(name)

# Load LoRA configuration
peft_config = LoraConfig(
  lora_alpha=lora_alpha,
  lora_dropout=lora_dropout,
  r=lora_r,
  bias="none",
  task_type="CAUSAL_LM",
  #target_modules=["q_proj", "v_proj"] # also ["o_proj", "k_proj"] ?
)

print(base_model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps

### Caricamento del modello StarCoder

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
starcoder = AutoModelForCausalLM.from_pretrained(
  starcoder_name,
  quantization_config=bnb_config,
  device_map=device_map
)
starcoder.eval()
# Load base tokenizer
tokenizer_starcoder = AutoTokenizer.from_pretrained(starcoder_name, trust_remote_code=True)
print(tokenizer_starcoder.pad_token, tokenizer_starcoder.eos_token)
tokenizer_starcoder.pad_token = tokenizer_starcoder.eos_token
print(tokenizer_starcoder.pad_token, tokenizer_starcoder.eos_token)
tokenizer_starcoder.padding_side = 'right'

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

None <|endoftext|>
<|endoftext|> <|endoftext|>


### Caricamento del modello fine-tuned da GitHub (per la valutazione)

In [ ]:
#Da Git:
!apt-get install git
!rm -rf FineTuningAI
!git clone https://github.com/matteraggi/FineTuningAI.git

#Dal Drive (cambiare path manualmente):
#from google.colab import drive
#drive.mount('/content/drive')
#!cp -r /content/drive/MyDrive/checkpoint_800 /content/

# Percorso alla directory del modello
checkpoint_path = "/content/FineTuningAI/models/DS_Finetuned_6"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)
# Il warning di sliding window è un false warning: https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/discussions/27

# Load (partially) finetuned model
finetuned_model = AutoModelForCausalLM.from_pretrained(
  checkpoint_path,
  device_map=device_map,
  quantization_config=bnb_config,
)
finetuned_model.eval()
# Load tokenizer from checkpoint
tokenizer_finetuned = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer_finetuned.pad_token = tokenizer_finetuned.eos_token
tokenizer_finetuned.padding_side = 'right'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Cloning into 'FineTuningAI'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 393 (delta 3), reused 4 (delta 1), pack-reused 382 (from 2)
Receiving objects: 100% (393/393), 679.45 MiB | 34.74 MiB/s, done.
Resolving deltas: 100% (217/217), done.
Updating files: 100% (81/81), done.


In [ ]:
# Ultime configurazioni per il modello
finetuned_model.config.use_cache = False
finetuned_model.config.pretraining_tp = 1
if gradient_checkpointing:
  finetuned_model.gradient_checkpointing_enable()  # Attiva gradient checkpointing per ridurre l'uso di memoria

# for name, module in finetuned_model.named_modules():
#    print(name)

# Load LoRA configuration
peft_config = LoraConfig(
  lora_alpha=lora_alpha,
  lora_dropout=lora_dropout,
  r=lora_r,
  bias="none",
  task_type="CAUSAL_LM",
  #target_modules=["q_proj", "v_proj"] # o ["o_proj", "k_proj"] ?
)

print(finetuned_model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1536, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=1536, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=1536, o

### Caricamento e tokenizzazione dataset

**DATASET LOAD AND PREPROCESSING**

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
# Use only a random subset of the dataset 'dataset_range'-wide
dataset = dataset.shuffle(seed=42).select(range(dataset_range))

# # Split the subset in training and validation
# split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = split_dataset["train"]
# val_dataset = split_dataset["test"]

# Use the full dataset for training only
train_dataset = dataset
# Per evitare OOM, è stata rimossa la validazione. Alla fine il codice verrà sempre valutato con humaneval.

In [ ]:
# Funzione che ritaglia solo la funzione dal testo di response

import re

def extract_code_from_response(text):
    # Cerca il primo blocco ```python ... ```
    match = re.search(r"```(?:python)?\n(.*?)```", text, re.DOTALL) # Tre backtick, 'python' opzionale, '\n', prendi il blocco, tre backtick. re.DOTALL per proseguire alle nuove linee
    if match:
        return match.group(1).strip() # Il contenuto catturato, stripped
    else:
        print('fallback!')  # DEBUGGING. Non dovrebbe printare nulla
        return text.strip() # Fallback

def clean_response(example):
    example["response"] = extract_code_from_response(example["response"])
    return example

dataset = dataset.map(clean_response)

In [ ]:
# Check dataset after cutting functions from responses

#print dataset columns
print(dataset.column_names)
#print dataset example
print(dataset[0])
#print dataset example with instruction and seed
print(f"Instruction: {dataset[1]['instruction']}")
print(f"Response: {dataset[1]['response']}")

['fingerprint', 'sha1', 'seed', 'response', 'concepts', 'prompt', 'instruction', 'id']
{'fingerprint': None, 'sha1': 'a1ea8d1a52a57874b576afc6a4c45e7d624d409e', 'seed': 'def box_enum(typ, val, c):\n    """\n    Fetch an enum member given its native value.\n    """\n    valobj = c.box(typ.dtype, val)\n    # Call the enum class with the value object\n    cls_obj = c.pyapi.unserialize(c.pyapi.serialize_object(typ.instance_class))\n    return c.pyapi.call_function_objargs(cls_obj, (valobj,))', 'response': 'def serialize_objects(objects):\n    """\n    Serialize a list of objects into a list of dictionaries.\n    Each dictionary represents the object\'s class name and attributes.\n    """\n    serialized_objects = []\n    for obj in objects:\n        serialized_object = {\n            "class_name": obj.__class__.__name__,\n            "attributes": {}\n        }\n        for attr_name, attr_value in obj.__dict__.items():\n            attr_dict = serialize_value(attr_value)\n            seri

In [ ]:
from statistics import mean, median

lengths = []

for ex in dataset:
    # Ricostruisci l'input esattamente come nel preprocessing
    full_text = f"{ex['instruction'].strip()}\n{ex['response'].strip()}"  # strip è opzionale
    # Tokenizzazione completa con special tokens (esattamente come nel preprocessing)
    input_ids = tokenizer_base(full_text, truncation=False)["input_ids"]
    #input_ids = tokenizer_finetuned(full_text, truncation=False)["input_ids"]
    lengths.append(len(input_ids))

# Analisi
max_len = max(lengths)
avg_len = mean(lengths)
med_len = median(lengths)
over_512 = sum(l > 512 for l in lengths) / len(lengths) * 100
over_1024 = sum(l > 1024 for l in lengths) / len(lengths) * 100
over_2048 = sum(l > 2048 for l in lengths) / len(lengths) * 100

print(f"Lunghezza massima: {max_len} token")
print(f"Lunghezza media: {avg_len:.2f} token")
print(f"Lunghezza mediana: {med_len} token")
print(f"Percentuale > 512 token: {over_512:.2f}%")
print(f"Percentuale > 1024 token: {over_1024:.2f}%")
print(f"Percentuale > 2048 token: {over_2048:.2f}%")

Lunghezza massima: 1340 token
Lunghezza media: 192.82 token
Lunghezza mediana: 174.0 token
Percentuale > 512 token: 1.25%
Percentuale > 1024 token: 0.10%
Percentuale > 2048 token: 0.00%


In [ ]:
# Funzione di preprocessing degli input
def preprocess_function(examples):
    input_ids_list = []
    label_ids_list = []
    attention_mask_list = []

    for instr, resp in zip(examples["instruction"], examples["response"]):
        prompt = instr.strip() + "\n" # Aggiungi uno /n alla fine di ogni insturction
        completion = resp.strip()     # In questo dataset lo strip è ridondante, ma non fa male

        # tokenizzazione separata di prompt e codice, senza token speciali
        prompt_ids = tokenizer_base(prompt, add_special_tokens=False)["input_ids"]
        completion_ids = tokenizer_base(completion, add_special_tokens=False)["input_ids"]

        # Li uniamo e 'svalutiamo' l'istruzione stessa per fare in modo che il modello impari solo a generare codice
        input_ids = prompt_ids + completion_ids
        labels = [-100] * len(prompt_ids) + completion_ids  # -100 label mask standard di huggingface per escludere token

        # Padding/truncation
        if len(input_ids) > max_seq_length:     # Tronca eventuali input troppo lunghi
            input_ids = input_ids[:max_seq_length]
            labels = labels[:max_seq_length]
        else:                                   # Aggiungi padding per arrivare a maxseq (e svaluta i token di padding)
            pad_len = max_seq_length - len(input_ids)
            input_ids += [tokenizer_base.pad_token_id] * pad_len
            labels += [-100] * pad_len

        attention_mask = [1 if token != tokenizer_base.pad_token_id else 0 for token in input_ids] # 1 per token validi, 0 per padding (in DS, padding=eos)

        input_ids_list.append(input_ids)
        label_ids_list.append(labels)
        attention_mask_list.append(attention_mask)

    return {
        "input_ids": input_ids_list,  # Frasi tokenizzate, input del modello
        "labels": label_ids_list, # Maschera corretta per supervisionare solo la parte target
        "attention_mask": attention_mask_list # Maschera per nascondere i padding
    }

# Applica il preprocessing
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
#tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

### Training (Fine-tuning)

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
  output_dir=output_dir,
  num_train_epochs=num_train_epochs,
  per_device_train_batch_size=per_device_train_batch_size,
  gradient_accumulation_steps=gradient_accumulation_steps,
  optim=optim,
  # eval_strategy=eval_strategy,  #eval
  # eval_steps=eval_steps,        #eval
  save_steps=save_steps,
  logging_steps=logging_steps,
  learning_rate=learning_rate,
  weight_decay=weight_decay,
  fp16=fp16,
  bf16=bf16,
  max_grad_norm=max_grad_norm,
  max_steps=max_steps,
  warmup_ratio=warmup_ratio,
  group_by_length=group_by_length,
  lr_scheduler_type=lr_scheduler_type,
  report_to="tensorboard",
  gradient_checkpointing=gradient_checkpointing,
  label_names=["labels"], # Risolve: No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
)

# Train the model with the modified configuration
trainer = SFTTrainer(
  model=base_model,
  train_dataset=tokenized_train_dataset,
  # eval_dataset=tokenized_val_dataset,  #eval
  peft_config=peft_config,
  args=training_arguments,
)

Truncating train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
# Start training

# Training da zero
trainer.train()

# Training se già parzialmente allenato
#trainer.train(resume_from_checkpoint=checkpoint_path)

Step,Training Loss
100,1.665300
200,0.229600
300,0.224300
400,0.216100
500,0.217400
600,0.208700
700,0.206100
800,0.209900
900,0.201500
1000,0.209700


TrainOutput(global_step=1500, training_loss=0.3051934852600098, metrics={'train_runtime': 10163.933, 'train_samples_per_second': 1.181, 'train_steps_per_second': 0.148, 'total_flos': 1.14457602686976e+17, 'train_loss': 0.3051934852600098})

### Salvataggio del modello / checkpoint

**Salva il modello**

In [ ]:
# Save the fine-tuned model
#trainer.model.save_pretrained(new_model)
trainer.save_model("./results/DS_Finetuned_6")

**Salva il checkpoint/modello pushandolo su GitHub**

In [ ]:
from google.colab import userdata
!apt-get install git
!git config --global user.email {userdata.get('GitEmail')}
!git config --global user.name {userdata.get('GitUsername')}
!git clone https://github.com/matteraggi/FineTuningAI.git
!cd FineTuningAI  # Go to the *existing* FineTuningAI directory
!mv results/DS_Finetuned_6 FineTuningAI/models/DS_Finetuned_6/
!cd FineTuningAI && git add models/DS_Finetuned_6
!cd FineTuningAI && git commit -m "Saved 'DS_Finetuned_6', uses cut responses."
!git config --global credential.helper store
!cd FineTuningAI && git push https://{userdata.get('PAT')}@github.com/matteraggi/FineTuningAI.git main

**Salva il checkpoint/modello su Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/results/DS_Finetuned_6 /content/drive/MyDrive/

### Valutazione HumanEval

**Caricare il benchmark HumanEval**

In [ ]:
SYSTEM_PROMPT = (
    "# Implement the following Python function.\n"
    "# Return only the code of the function. Do not include comments like TODO or placeholders.\n"
    "# Do not leave the function body empty or incomplete.\n"
    "# If examples or docstrings are present, use them to infer the logic.\n"
    "# Make sure the function is correct and complete.\n\n"

)
#SYSTEM_PROMPT = """Complete the following python function as described in its description.
#                    In said description, there are also lines beginning with >>> that symbolize execution examples, useful for understanding how the function should work.\n"""

In [ ]:
# Configurazione
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device in use:", device) #DEBUG

# === Funzione di generazione codice da parte del modello ===
def generate_code(model, prompt, tokenizer):
    try:
        inputs = tokenizer(prompt, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                temperature=temperature,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as e:
        print(f"Errore in generate_code(): {e}")
        return "Errore durante la generazione"

Confronto tra Starcoder e DeepSeek finetunato (SYSTEM_PROMPT + funzione)



In [ ]:
import torch
import json
import gc
import random
import pandas as pd
from tabulate import tabulate
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Caricamento del dataset HumanEval e selezione casuale di 10 esempi
#humaneval = load_dataset("openai/openai_humaneval", split="test")
#sampled_humaneval = random.sample(list(humaneval), 2)
#with open("fixed_sampled_humaneval.json", "w") as f:
 #   json.dump(sampled_humaneval, f, indent=4)
with open("fixed_sampled_humaneval.json", "r") as f:
    sampled_humaneval = json.load(f)
starcoder.eval()
finetuned_model.eval()

# Valutazione dei modelli
starcoder_results = []
fine_tuned_results = []

for i, example in enumerate(sampled_humaneval):
    partial_prompt = example['prompt']
    prompt = SYSTEM_PROMPT + partial_prompt

    starcoder_code = generate_code(starcoder, prompt, tokenizer_starcoder)
    starcoder_results.append({"prompt": prompt, "code": starcoder_code})

    fine_tuned_code = generate_code(finetuned_model, prompt, tokenizer_finetuned)
    fine_tuned_results.append({"prompt": prompt, "code": fine_tuned_code})

# Salvataggio dei risultati in JSON
results = {"starcoder_results": starcoder_results, "fine_tuned_results": fine_tuned_results}

with open("model_results.json", "w") as f:
    json.dump(results, f, indent=4)

# Creazione e stampa tabella
df = pd.DataFrame([
    {"Prompt": b['prompt'], "Non-Finetuned Code": b['code'], "Finetuned Code": f['code']}
    for b, f in zip(starcoder_results, fine_tuned_results)
])

files.download("model_results.json")

print(tabulate(df, headers="keys", tablefmt="grid", numalign="left", stralign="left"))

device in use: cuda
Processed example 1/10


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+----+-------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | Prompt                                                                        | Non-Finetuned Code                                                                                      | Finetuned Code                                                                                                                                                                                                                                            |
+====+===============================================================================+========================

Confronto tra Starcoder e DeepSeek finetunato (solo docstring)

In [ ]:
from datasets import load_dataset
import re
import json
import torch
import gc
import pandas as pd
from tabulate import tabulate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from google.colab import files

# Configurazione
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device in use:", device)

# Caricamento del dataset HumanEval
humaneval = load_dataset("openai/openai_humaneval", split="test")


###############################################################################
#sampled_humaneval = random.sample(list(humaneval), 10)

# Salva il set di test una sola volta
#with open("fixed_sampled_humaneval.json", "w") as f:
    #json.dump(sampled_humaneval, f, indent=4)
# Carica il dataset salvato
with open("fixed_sampled_humaneval.json", "r") as f:
    sampled_humaneval = json.load(f)
###############################################################################

# starcoder.eval() ???

# === Funzione per estrarre la docstring dal codice ===
def estrai_docstring(code):
    """Estrae il contenuto tra triple virgolette singole o doppie."""
    match = re.search(r'("""|\'\'\')(.*?)(\1)', code, re.DOTALL)
    return match.group(2).strip() if match else ""

# === Valutazione ===
starcoder_results = []
fine_tuned_results = []

for i, example in enumerate(sampled_humaneval):
    docstring = estrai_docstring(example["prompt"])

    starcoder_code = generate_code(base_model, docstring, tokenizer_base)
    starcoder_results.append({"prompt": docstring, "code": starcoder_code})

    fine_tuned_code = generate_code(finetuned_model, docstring, tokenizer_finetuned)
    fine_tuned_results.append({"prompt": docstring, "code": fine_tuned_code})

# === Salva risultati ===
results = {"starcoder_results": starcoder_results, "fine_tuned_results": fine_tuned_results}
with open("model_results.json", "w") as f:
    json.dump(results, f, indent=4)

# === Tabella riassuntiva ===
df = pd.DataFrame([
    {"Prompt": b['prompt'], "Non-Finetuned Code": b['code'], "Finetuned Code": f['code']}
    for b, f in zip(starcoder_results, fine_tuned_results)
])
files.download("model_results.json")
print(tabulate(df, headers="keys", tablefmt="grid", numalign="left", stralign="left"))


device in use: cuda


NameError: name 'non_finetuned_model' is not defined

### Salvataggio / caricamento dei risultati

**SAVE IN DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Save as JSON in Drive
with open("/content/drive/MyDrive/model_results.json", "w") as f:
    json.dump(results, f, indent=4)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAVE IN GIT**

In [ ]:
from google.colab import userdata
import os

# Install git (se non già installato)
!apt-get install git -qq

# Configura Git (modo corretto per Colab)
git_email = userdata.get('GitEmail')
git_username = userdata.get('GitUsername')
pat = userdata.get('PAT')

!git config --global user.email "{git_email}"
!git config --global user.name "{git_username}"

# Clona il repository (se non esiste già o forza il reclone)
if not os.path.exists('FineTuningAI'):
    !git clone https://{pat}@github.com/matteraggi/FineTuningAI.git
else:
    !rm -rf FineTuningAI
    !git clone https://{pat}@github.com/matteraggi/FineTuningAI.git

# Copia i risultati
!cp model_results.json FineTuningAI/results/

# Commit e push
%cd FineTuningAI
!git add results/model_results.json
!git commit -m "Save model results $(date +'%Y-%m-%d %H:%M:%S')"
!git push origin main
%cd ..

Cloning into 'FineTuningAI'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 241 (delta 9), reused 15 (delta 3), pack-reused 212 (from 1)
Receiving objects: 100% (241/241), 364.12 MiB | 33.41 MiB/s, done.
Resolving deltas: 100% (124/124), done.
Updating files: 100% (46/46), done.
/content/FineTuningAI
[main 424aab1] Save model results 2025-04-06 11:16:46
 1 file changed, 86 insertions(+), 86 deletions(-)
 rewrite results/model_results.json (98%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 4.15 KiB | 4.15 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/matteraggi/FineTuningAI.git
   57a9714..424aab1  main -> main
/content


**LOAD FROM DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import json

# Load JSON file from Drive
with open("/content/drive/MyDrive/model_results.json", "r") as f:
    results = json.load(f)

# Now results['base_results'] and results['fine_tuned_results'] hold the data
base_results = results['base_results']
fine_tuned_results = results['fine_tuned_results']


Mounted at /content/drive


**LOAD FROM GIT**

In [ ]:
import json
!apt-get install git
!git clone https://github.com/matteraggi/FineTuningAI.git

with open("FineTuningAI/results/model_results.json", "r") as f:
    results = json.load(f)

base_results = results['base_results']
fine_tuned_results = results['fine_tuned_results']

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
fatal: destination path 'FineTuningAI' already exists and is not an empty directory.


### Confronto tra modelli ("*Complex checking*")

In [ ]:
!pip install pylint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.6/522.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.4/275.4 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 9.3 MB/s eta 0:00:00


In [ ]:
# Creazione di un set di confronto
humaneval = load_dataset("openai/openai_humaneval", split="test")
sampled_humaneval = random.sample(list(humaneval), 1)
with open("fixed_sampled_humaneval.json", "w") as f:    # Salva il set di confronto per evitare di ricrearlo ogni volta
    json.dump(sampled_humaneval, f, indent=4)

README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

**FUNZIONI DI CONFRONTO**

In [ ]:
import ast
import subprocess
import tempfile
import zss
import re
import os
from radon.complexity import cc_visit
import textwrap
import re


def estrai_docstring( code):
  match = re.search(r'("""|\'\'\')(.*?)(\1)', code, re.DOTALL)
  return match.group(2).strip() if match else ""

def code_similarity(code1, code2):
  tokens1 = set(re.findall(r'\b\w+\b', code1))
  tokens2 = set(re.findall(r'\b\w+\b', code2))
  if not tokens1 or not tokens2:
    return 0.0
  intersection = tokens1 & tokens2
  union = tokens1 | tokens2
  return len(intersection) / len(union)

def run_pylint_analysis( code: str) -> dict:
  results = {
    "errors": 0,
    "warnings": 0,
    "conventions": 0,
    "refactors": 0,
    "messages": []
  }
  with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as temp_file:
    temp_file.write(code)
    temp_file_path = temp_file.name
  try:
    completed = subprocess.run(
        ["pylint", "--disable=all", "--enable=E,W,C,R", "--score=n", temp_file_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    for line in completed.stdout.splitlines():
      results["messages"].append(line)
      if re.search(r'\bE\d{4}\b', line):
        results["errors"] += 1
      elif re.search(r'\bW\d{4}\b', line):
        results["warnings"] += 1
      elif re.search(r'\bC\d{4}\b', line):
        results["conventions"] += 1
      elif re.search(r'\bR\d{4}\b', line):
        results["refactors"] += 1
  except Exception as e:
    results["messages"].append(f"Errore durante analisi pylint: {str(e)}")
  finally:
    os.remove(temp_file_path)
  return results

def extract_python_code( generated: str, prompt: str) -> str:
  # 1. Rimuove prompt (se presente all'inizio)
  generated = generated.strip()
  if generated.startswith(prompt.strip()):
    generated = generated[len(prompt):].strip()

  # 2. Cerca un blocco markdown ```python ... ```
  match = re.search(r"```(?:python)?(.*?)```", generated, re.DOTALL)
  if match:
    code_block = match.group(1).strip()
    # Verifica se contiene un 'def'; se sì, usalo
    if 'def ' in code_block:
        return code_block

  # 3. Se non c'è blocco valido, cerca direttamente un 'def' nel testo
  lines = generated.splitlines()
  def_index = next((i for i, line in enumerate(lines) if line.strip().startswith("def ")), None)
  if def_index is not None:
    return "\n".join(lines[def_index:]).strip()

  # 4. Fallback: restituisce comunque tutto ciò che resta
  return generated.strip()

def preprocess_code( code: str) -> str:
    if not code:
        return ""
    matches = re.findall(r'(def\s+\w+\(.*?\):.*?(?=\n\s*def\s|\Z))', code, re.DOTALL)
    code = matches[0] if matches else code
    code = re.sub(r'#.*', '', code)
    code = re.sub(r'\.\.\..*', '', code)
    code = re.sub(r'\s+\n', '\n', code)
    return code.strip()

def is_valid_python( code: str) -> bool:
  try:
    ast.parse(code)
    return True
  except:
    return False

def safe_parse( code):
  try:
    return ast.parse(code)
  except:
    return None

def calculate_complexity( code: str) -> float:
  try:
    blocks = cc_visit(code)
    return sum(b.complexity for b in blocks) / len(blocks) if blocks else 0
  except:
    return 0

def ast_to_node( node):
    if isinstance(node, ast.AST):
        children = [ast_to_node(child) for child in ast.iter_child_nodes(node)]
        return zss.Node(type(node).__name__, children)
    return zss.Node(str(node))

def count_control_structures( code: str) -> int:
  try:
    tree = safe_parse(code)
    if not tree:
      return 0
    return sum(isinstance(n, (ast.If, ast.For, ast.While)) for n in ast.walk(tree))
  except:
    return 0

def has_function_def( code: str) -> bool:
  try:
    tree = safe_parse(code)
    if not tree:
      return False
    return any(isinstance(n, ast.FunctionDef) for n in ast.walk(tree))
  except:
    return False

def is_meaningfully_implemented( code: str) -> bool:
  tree = safe_parse(code)
  if not tree:
    return False
  for node in ast.walk(tree):
    if isinstance(node, ast.Return) and node.value is not None:
        return True
    if isinstance(node, (ast.Yield, ast.Raise)):
        return True
  return False

def evaluate_code( code: str, canonical: str = None) -> dict:
  clean_code = code.strip()
  tree = safe_parse(clean_code)
  similarity_score = code_similarity(clean_code, canonical)
  return {
    "syntactic_correct": is_valid_python(clean_code),
    "avg_complexity": calculate_complexity(clean_code),
    "code_length": len(clean_code),
    "has_function_def": has_function_def(clean_code),
    "has_return": "return" in clean_code.lower(),
    "implementation_complete": is_meaningfully_implemented(tree),
    "code_similarity": similarity_score,
    **run_pylint_analysis(clean_code)
  }


# Funzioni per eseguire il confronto

def run_evaluation(useSystemPrompt):
    with open("fixed_sampled_humaneval.json", "r") as f:
        sampled_humaneval = json.load(f)
    results = {"base": [], "starcoder": [], "fine_tuned": []}
    for idx, example in enumerate(sampled_humaneval):
      canonical = example['canonical_solution']
      if(useSystemPrompt):  # Caso system prompt + funzione
        partial_prompt = example['prompt']
        prompt = SYSTEM_PROMPT + partial_prompt
      else:                 # Caso docstring
        prompt = estrai_docstring(example["prompt"])
      
      raw_base = generate_code(base_model, prompt, tokenizer_base)
      base_code = extract_python_code(raw_base, prompt)
      base_metrics = evaluate_code(base_code, canonical)

      raw_starcoder = generate_code(starcoder, prompt, tokenizer_starcoder)
      starcoder_code = extract_python_code(raw_starcoder, prompt)
      starcoder_metrics = evaluate_code(starcoder_code, canonical)

      raw_finetuned = generate_code(finetuned_model, prompt, tokenizer_finetuned)
      fine_tuned_code = extract_python_code(raw_finetuned, prompt)
      fine_tuned_metrics = evaluate_code(fine_tuned_code, canonical)

      results["base"].append({
            "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
            "code": base_code[:200] + "..." if len(base_code) > 200 else base_code,
            **base_metrics
        })
      results["starcoder"].append({
            "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
            "code": starcoder_code[:200] + "..." if len(starcoder_code) > 200 else starcoder_code,
            **starcoder_metrics
        })
      results["fine_tuned"].append({
            "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
            "code": fine_tuned_code + "..." if len(fine_tuned_code) > 200 else fine_tuned_code,
            **fine_tuned_metrics
      })
      gc.collect()
    analyze_results(results)

def analyze_results( results):
    base_df = pd.DataFrame(results["base"])
    fine_tuned_df = pd.DataFrame(results["fine_tuned"])
    starcoder_df = pd.DataFrame(results["starcoder"])
    print("\n=== Metriche Aggregate ===")
    agg_metrics = [
        ["Sintatticamente Corretto", base_df["syntactic_correct"].mean(), starcoder_df["syntactic_correct"].mean(), fine_tuned_df["syntactic_correct"].mean()],
        ["Errori pylint", base_df["errors"].mean(), starcoder_df["errors"].mean(), fine_tuned_df["errors"].mean()],
        ["Warning pylint", base_df["warnings"].mean(), starcoder_df["warnings"].mean(), fine_tuned_df["warnings"].mean()],
        ["Similarità del codice", base_df["code_similarity"].mean(), starcoder_df["code_similarity"].mean(), fine_tuned_df["code_similarity"].mean()],
        ["Complessità Ciclomatica", base_df["avg_complexity"].mean(), starcoder_df["avg_complexity"].mean(), fine_tuned_df["avg_complexity"].mean()],
        ["Implementazione Completa", base_df["implementation_complete"].mean(), starcoder_df["implementation_complete"].mean(), fine_tuned_df["implementation_complete"].mean()]
    ]
    print(tabulate(agg_metrics, headers=["Metrica", "Base", "Starcoder", "Fine-Tuned"], floatfmt=".2f", tablefmt="grid"))


# Setup per l'esecuzione

#os.environ["HF_ALLOW_CODE_EVAL"] = "1"
base_model.eval()
starcoder.eval()
finetuned_model.eval()

**Confronto con SYSTEM_PROMPT + funzione tra tutti i modelli**

In [ ]:
SYSTEM_PROMPT = (
    "# Implement the following Python function.\n"
    "# Return only the code of the function. Do not include comments like TODO or placeholders.\n"
    "# Do not leave the function body empty or incomplete.\n"
    "# If examples or docstrings are present, use them to infer the logic.\n"
    "# Make sure the function is correct and complete.\n\n"
)

run_evaluation(True)    # Con System Prompt + Funzione


================ ESEMPIO 1/30 ================

📘 Codice generato - Base Model:

None 


📗 Codice generato - Starcoder:


📙 Codice generato - Fine-Tuned Model:


================ ESEMPIO 2/30 ================

📘 Codice generato - Base Model:

None 


📗 Codice generato - Starcoder:


📙 Codice generato - Fine-Tuned Model:


================ ESEMPIO 3/30 ================

📘 Codice generato - Base Model:

None 


📗 Codice generato - Starcoder:


📙 Codice generato - Fine-Tuned Model:


================ ESEMPIO 4/30 ================

📘 Codice generato - Base Model:

None 


📗 Codice generato - Starcoder:


📙 Codice generato - Fine-Tuned Model:


================ ESEMPIO 5/30 ================

📘 Codice generato - Base Model:

None 


📗 Codice generato - Starcoder:


📙 Codice generato - Fine-Tuned Model:


================ ESEMPIO 6/30 ================

📘 Codice generato - Base Model:

None 


📗 Codice generato - Starcoder:


📙 Codice generato - Fine-Tuned Model:


================ ESEMPIO 7/

**Confronto con docstring tra tutti i modelli**

In [ ]:
run_evaluation(False)   # Con Docstring


=== Metriche Aggregate ===
+--------------------------+--------+-------------+--------------+
| Metrica                  |   Base |   Starcoder |   Fine-Tuned |
+==========================+========+=============+==============+
| Sintatticamente Corretto |   0.00 |        0.20 |         0.33 |
+--------------------------+--------+-------------+--------------+
| Errori pylint            |   1.00 |        1.07 |         0.87 |
+--------------------------+--------+-------------+--------------+
| Warning pylint           |   0.00 |        0.20 |         0.10 |
+--------------------------+--------+-------------+--------------+
| Similarità del codice    |   0.05 |        0.13 |         0.22 |
+--------------------------+--------+-------------+--------------+
| Complessità Ciclomatica  |   0.00 |        0.60 |         0.97 |
+--------------------------+--------+-------------+--------------+
| Implementazione Completa |   0.00 |        0.10 |         0.30 |
+--------------------------+------